In [81]:
'''
# importing the "tarfile" module
import tarfile
  
# open file
file = tarfile.open('cabspottingdata.tar.gz')
  
# extracting file
file.extractall('./preprocessing')
  
file.close()
'''

'\n# importing the "tarfile" module\nimport tarfile\n  \n# open file\nfile = tarfile.open(\'cabspottingdata.tar.gz\')\n  \n# extracting file\nfile.extractall(\'./preprocessing\')\n  \nfile.close()\n'

In [82]:
# Read Text Files with Pandas using read_csv()
  
# importing pandas
import pandas as pd
  
import glob
taxis = []
files = glob.glob("./cabspottingdata/new_*.txt")
for file in files:
    df = pd.read_csv(file, sep=" ", header=None, names=["Latitude", "Longitude", "Fare", "Timestamp"], dtype=None)
    df.drop(labels='Fare', axis=1, inplace=True)
    df['Timestamp'] = df['Timestamp'].astype(float)
    taxis.append(df)

In [83]:
# Transform from unix timestamp to time since start of simulation
minTime = taxis[0]["Timestamp"].min()

# print("taxis[0].head()", taxis[0].head())
# print("minTime: ", minTime)
for df in taxis:
    # print("minTime: ", minTime)
    # print("minTime: ", minTime)
    # print("df[\"Timestamp\"].min()", df["Timestamp"].min())
    minTime = min(minTime, df["Timestamp"].min())
# print(f"minTime: {minTime}")

for df in taxis:
    df['Timestamp'] = df['Timestamp'].apply(lambda x: x - minTime) # 1210982400)


In [84]:
positions = []
timestamps = []

from tqdm.notebook import tqdm 
for taxi in tqdm(taxis, desc="Iterating through taxis: "):
    this_taxi_positions = []
    this_taxi_timestamps = []
    for i in range(len(taxi)):
        this_taxi_positions.append([taxi["Longitude"][i], taxi["Latitude"][i]])
        this_taxi_timestamps.append(taxi["Timestamp"][i])
    positions.append(this_taxi_positions)
    timestamps.append(this_taxi_timestamps[::-1])
positions = list(map(lambda elem: elem[:50], positions))
timestamps = list(map(lambda elem: elem[:50], timestamps))

Iterating through taxis:   0%|          | 0/536 [00:00<?, ?it/s]

In [85]:
outputobj = [] 
currentVendor = 0
for i in range(len(positions)):
    outputobj.append({ "vendor": currentVendor, "path": positions[i], "timestamps": timestamps[i] })
    currentVendor = (currentVendor + 1) % 3

In [86]:
import json

print(len(outputobj))
with open("../trips/taxis.json", "w") as file:
    file.write(json.dumps(outputobj))

536
